<div style="text-align: right" align="right"><i>Peter Norvig, December 2025</i></div>

# Advent of Code 2025

I  enjoy doing the [**Advent of Code**](https://adventofcode.com/) (AoC) programming puzzles, so here we go for 2025!  This year I will be doing something different: I will solve each problem, and then [**in another notebook**](Advent-2025-AI.ipynb) I will ask an AI Large Language Model to solve the same problemm, record the response, and comment on it. I'll alternate between Gemini, Claude, and ChatGPT.

# Day 0

I'm glad that [@GaryGrady](https://mastodon.social/@garygrady) is providing cartoons:

<a href="https://x.com/garyjgrady"><img src="https://pbs.twimg.com/media/Gdp709FW8AAq2_m?format=jpg&name=medium" width=400 alt="Gary Grady cartoon"></a>

I start by loading up my [**AdventUtils.ipynb**](AdventUtils.ipynb) notebook (same as last time except for the `current_year`). On each day I will first parse the input (with the help of my `parse` utility function), then solve Part 1 and Part 2 (recording the correct answer with my `answer` function).

In [1]:
%run AdventUtils.ipynb
current_year = 2025

# [Day 1](https://adventofcode.com/2025/day/1): Secret Entrance

On Day 1 we meet an elf and learn that our task is to finish decorating the North Pole by December 12th. There will be challenges along the way. Today we need to unlock a safe. The safe has a dial with 100 numbers. Our input for today is a sequence of left and right rotations; for example "R20" means move the dial right by 20 numbers. I'll use my `parse` utility function to parse each line of the input as an integer, after replacing each 'L' with a minus sign and each 'R' with a plus sign:

In [2]:
rotations = parse(day=1, parser=lambda line: int(line.replace('L', '-').replace('R', '+')))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 4780 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
L20
L13
L16
L16
L29
L7
L48
L48
...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 4780 ints:
────────────────────────────────────────────────────────────────────────────────────────────────────
-20
-13
-16
-16
-29
-7
-48
-48
...


<img src="https://files.mastodon.social/media_attachments/files/115/646/343/679/448/846/original/428b312ca88f62c4.jpg" width=400 alt="Gary Grady cartoon">

### Part 1: How many times is the dial left pointing at 0 after any rotation in the sequence?

Initially the safe's arrow is pointing at 50, and then we apply the rotations in order. We are asked how many of the rotations leave the dial pointing at 0. The `itertools.accumulate` function yields running totals of its input sequence, so we just have to count (quantify) how many times the running total of the rotations is 0 mod 100:

In [3]:
def count_zeros(numbers, dial=100) -> int:
    """How many zeros (modulo `dial`) in the running partial sums of the numbers?"""
    return quantify(total % dial == 0 for total in accumulate(numbers, initial=50))

Here's the process I repeat for each puzzle: I ran `count_zeros(rotations)`, submitted the answer to AoC, and once I saw it was correct, I recorded the answer as follows:

In [4]:
answer(puzzle=1.1, solution=1182, code=lambda: 
       count_zeros(rotations))

Puzzle  1.1:   .0008 seconds, answer 1182            ok

### Part 2: How many times does the dial point to 0 at any time?

For Part 2 we need to count both when a rotation ends up at 0 and when the arrow passes 0 in the middle of the rotation. For example, if the arrow points to 95, then only a "R5" or a "L95" would register a 0 in Part 1, but now a rotation of "R10" would count because it passes 0 (as would any rotation of "R5" or larger, or "L95" or larger). 

I'll start with a simple but slow approach: treat a rotation of, say, -20 as 20 rotations of -1, and then use the same `count_zeros` function from part 1. (Note that `sign(r)` returns +1 for any positive input, and -1 for any negative input.)

In [5]:
answer(1.2, 6907, lambda:
       count_zeros(sign(r) for r in rotations for _ in range(abs(r))))

Puzzle  1.2:   .1516 seconds, answer 6907            ok

I can speed this up by adding up the full-circle rotations separately from the partial rotations. That is, a rotation of "L995" does 9 complete circles of the dial (and thus 9 zero crossings), and then moves 95 more clicks (possibly crossing zero once more, depending on where we start). That gives us this:

In [6]:
answer(1.2, 6907, lambda:
       sum(abs(r) // 100 for r in rotations) +
       count_zeros(sign(r) for r in rotations for _ in range(abs(r) % 100)))

Puzzle  1.2:   .0489 seconds, answer 6907            ok

That's three times faster, but still a comparatively long run time for a Day 1 problem, so here's a faster method. I break each rotation down into a number of full circles and some remainder, then add the full circles to the count of zeros, and add one more if the remainder is at least as much as the distance to zero: 

In [7]:
def zero_clicks(rotations, position=50, dial=100) -> int:
    """How many times does any click cause the dial to point at 0?
    Count 1 if the rotation crosses the distance to 0,
    and for large rotations, count abs(r) // 100 more."""
    zeros = 0
    for r in rotations:
        full_circles, remainder = divmod(abs(r), dial)
        distance_to_0 = (dial - position if (r > 0 or position == 0) else position)
        zeros += full_circles + (1 if remainder >= distance_to_0 else 0)
        position = (position + r) % dial
    return zeros

In [8]:
answer(1.2, 6907, lambda:
       zero_clicks(rotations))

Puzzle  1.2:   .0010 seconds, answer 6907            ok

That's much faster, but the code is trickier, and indeed I initially had a **bug** in the `distance_to_0` computation: when the current position is 0 the distance should be 100: it takes a full rotation to get back to 0. My code initially claimed the distance was 0; adding `or position == 0` fixed that.

# [Day 2](https://adventofcode.com/2025/day/2): Gift Shop

Today we're in the North Pole gift shop, and are asked to help the elves identify invalid product IDs on the items there. We're giving a list of ranges of product IDs. Each range is a pair of integers separated by a dash, and the ranges are separated by commas:

In [9]:
id_ranges = parse(day=2, parser=positive_ints, sections=lambda text: text.split(','))

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 1 str:
────────────────────────────────────────────────────────────────────────────────────────────────────
990244-1009337,5518069-5608946,34273134-34397466,3636295061-3636388848,8613701-8663602,573252-68 ...
────────────────────────────────────────────────────────────────────────────────────────────────────
Parsed representation ➜ 35 tuples:
────────────────────────────────────────────────────────────────────────────────────────────────────
(990244, 1009337)
(5518069, 5608946)
(34273134, 34397466)
(3636295061, 3636388848)
(8613701, 8663602)
(573252, 688417)
(472288, 533253)
(960590, 988421)
...


<img src="https://files.mastodon.social/media_attachments/files/115/652/152/368/251/243/original/56e4ed8e5f24db96.jpg" width=400 alt="GaryJGrady cartoon">

### Part 1: What is the sum of the invalid IDs?

An invalid ID is one that consists of a digit sequence repeated twice. So 55, 6464 and 123123 are invalid. We're asked for the sum of the invalid IDs across all the ID ranges.

We could look at every number in each range and check if the first half of the number (as a string) is the same as the second half. How many checks would that be?

In [10]:
sum((hi - lo + 1) for lo, hi in id_ranges)

1990936

Only 2 million! So it would indeed be feasible to check every one. But I have a suspicion that Part 2 would make it infeasible, so I'll invest in a more efficient approach. For each ID range, instead of enumerating every number in the range and checking each one for validity, I will instead enumerate over the *first half* of the possible digit strings, and automatically generate invalid IDs by appending a copy of the first half to itself.

Suppose the range is 123456-223000.  I enumerate from 123 to 223, and for each one form generate an invalid ID:
[123123, 124124, 125125, ... 223223]. I then yield the IDs that are within the range; in this all but the first and the last. Altogether I only have to consider 101 IDs rather than 100,001. The algorithm scales with the square root of the size of the range, not with the size of the range itself.

In [75]:
def generate_invalids(lo: int, hi: int) -> Iterable[int]:
    """Yield all the invalid IDs between lo and hi inclusive.
    An ID is invalid if it consists of a digit sequence repeated twice."""
    lo_str = str(lo)
    start = int(lo_str[:max(1, len(lo_str) // 2)])
    for half in count_from(start):
        id = int(str(half) * 2)
        if lo <= id <= hi:
            yield id
        elif id > hi:
            return

assert list(generate_invalids(11, 22)) == [11, 22]

In [12]:
answer(2.1, 23560874270, lambda:
       sum(sum(generate_invalids(lo, hi)) for lo, hi in id_ranges))

Puzzle  2.1:   .0030 seconds, answer 23560874270     ok

### Part 2: What is the sum of the invalid IDs, under the new rules?

In Part 2 we discover that an ID should be considered invalid if it repeats a sequence of digits two *or more* times. So 111 (repeated three times), 12121212 (four times), and 222222 (six times) are all invalid. I'll rewrite `generate_invalids` to take an optional argument saying how many repeats we're looking for, and introduce  `generate_all_invalids` to try all possible repeat lengths:

In [13]:
def generate_invalids(lo: int, hi: int, repeat=2) -> Iterable[int]:
    """Yield all the invalid IDs between lo and hi inclusive
    that are formed from `repeat` sequences."""
    lo_str = str(lo)
    start = int(lo_str[:len(lo_str) // repeat] or 1)
    for i in count_from(start):
        id = int(str(i) * repeat)
        if lo <= id <= hi:
            yield id
        elif id > hi:
            return

def generate_all_invalids(lo: int, hi: int) -> Set[int]:
    """All invalid numbers in the range lo to hi inclusive,
    under the rules where 2 or more repeated digit sequences are invalid."""
    return {id for repeat in range(2, len(str(hi)) + 1)
               for id in generate_invalids(lo, hi, repeat)}
    
assert list(generate_invalids(11, 22)) == [11, 22]
assert list(generate_invalids(2121212118, 2121212124, 5)) == [2121212121]
assert list(generate_invalids(95, 115, 3)) == [111]
assert list(generate_all_invalids(95, 115)) == [99, 111]

Now verify that the answer for Part 1 still works, and go ahead and compute the answer for Part 2:

In [14]:
answer(2.1, 23560874270, lambda:
       sum(sum(generate_invalids(lo, hi)) for lo, hi in id_ranges))

Puzzle  2.1:   .0029 seconds, answer 23560874270     ok

In [15]:
answer(2.2, 44143124633, lambda:
       sum(sum(generate_all_invalids(lo, hi)) for lo, hi in id_ranges))

Puzzle  2.2:   .0038 seconds, answer 44143124633     ok

I had another **bug** here: initially I counted "222222" twice: once as 2 repeats of "222" and once as 3 repeats of "22". I changed the output of `generate_all_invalids` to be a set to fix that.

# [Day 3](https://adventofcode.com/2025/day/3): Lobby

Entering the lobby, we find that the elevators are offline. We might be able to fix the problem by turning on some batteries. There are multiple battery banks, each bank consisting of a sequence of batteries, each labeled with its *joltage*, a digit from 1 to 9. 

In [16]:
banks = parse(day=3)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 200 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
5353323523322232362334333433323333353233331313222372133133353643423323233323333534414523333432223242
6344544745655555456556556566665564538465555575558846455665837545764555554465564547547565544657585435
2246273372253242254243532252231242225522622633532222322234255122531222423531343223123232234213323424
6545643634344444495734739454433454439454355654483544243344534445434437426443854344454534654439534424
2356636643143433535443636338231745346538433576334436353176353333433532345344334224435234343644332536
3221311221443323323322222214632342232233222322333436263122265162212321261323142262212332322125216222
3336332333336335335324359336493238433441666379243536334165623214253384333323893933867663434332383763
323532125233243133222323243622253243222622322221323343285353532231

### Part 1: What is the maximum possible total output joltage?

We can turn on exactly two batteries in each bank, resulting in a two digit number which is the *joltage* of the bank. For example, given the bank "8647" we could choose to turn on the "8" and "7" to produce a joltage of 87.  The function `joltage` chooses the biggest first digit, and then the biggest second digit that follows the first digit. Note that the possible choices for the first digit exclude the last digit, because if we chose that, then we couldn't choose a second digit to follow. Note also I chose to return a string rather than an int; that seemed simpler within `joltage` but it does mean the caller might need to do a conversion.

In [17]:
def joltage(bank: str) -> str:
    """The maximum possible joltage by turning on 2 batteries in the bank."""
    choices = bank[:-1] # The first digit can't be the last character
    index = first(bank.index(d) for d in '987654321' if d in choices)
    return bank[index] + max(bank[index + 1:])

assert joltage("8647") == "87"
assert joltage("1119") == "19"

In [18]:
answer(3.1, 17085, lambda:
       sum(int(joltage(b)) for b in banks))

Puzzle  3.1:   .0004 seconds, answer 17085           ok

### Part 2: What is the new maximum possible total output joltage?

In Part 2 the elf hits the "joltage limit safety override" button, and we can now turn on 12 batteries per bank, resulting in a 12-digit joltage. What is the new maximum possible total joltage?

I will make a change to the function `joltage`, passing it the number of digits remaining to be chosen, *n*. The function stops when we get to 1 digit remaining, and recurses when there is more than one digit remaining. At each step we need to make sure the choice of first digit leaves *n*-1 digits for later choices.

In [19]:
def joltage(bank: str, n=2) -> str:
    """The maximum possible joltage by turning on `n` batteries in the bank."""
    if n == 1:
        return max(bank)
    else:
        choices = bank[:-(n - 1)]
        index = first(bank.index(d) for d in '987654321' if d in choices)
        return bank[index] + joltage(bank[index + 1:], n - 1)

assert joltage("811111111111119", 2)  == '89'
assert joltage("818181911112111", 5)  == '92111'
assert joltage("818181911112111", 12) == '888911112111'

I'll first make sure that the new version of `joltage` is backwards compatible, and then solve Part 2:

In [20]:
answer(3.1, 17085, lambda:
       sum(int(joltage(b)) for b in banks))

Puzzle  3.1:   .0005 seconds, answer 17085           ok

In [21]:
answer(3.2, 169408143086082, lambda:
       sum(int(joltage(b, 12)) for b in banks))

Puzzle  3.2:   .0022 seconds, answer 169408143086082 ok

# [Day 4](https://adventofcode.com/2025/day/4): Printing Department

The floor of the printing department is divided into squares. Many squares contain a roll of paper; other squares are empty. The day's input is a map of the floor, with `@` representing a roll of paper. I can handle that with the `Grid` class from my AdventUtils:

In [39]:
paper_grid = Grid(parse(day=4), directions=directions8)

────────────────────────────────────────────────────────────────────────────────────────────────────
Puzzle input ➜ 140 strs:
────────────────────────────────────────────────────────────────────────────────────────────────────
.@@@@@...@.@@@@@@@@@@.@@@@@@@.@.@.@@@@@@@@@@@@@..@.@@@.@@@@@@..@.@..@.@@...@.@@@@..@@@@....@@@.@ ...
.@@@@@.@....@.....@@@.@@.@@.@@@.@@@.@.@.@.@@@@@.@@.@@@@@.@@@@@@@@@@@..@@.@.@@.@@@.@@@@@@@@@@@..@ ...
@.@@@@.@@@@.@@@@..@@.@@@@@@@@.@@@@.@@@@.@@..@.@...@.@.@.@.@@..@@@@@.@.@.@@@@.@@@@@@@@@.@@@@..@@. ...
.@.....@.@@@..@.@@@.@..@@@@@..@@@.@@..@...@.@@@@.@@@.@.@@@@@@.@.@@@@@@@.@.@@@.@@@@@@...@@.@@..@. ...
@@@@@.@@@.@@@@@@@..@@.@.@@@..@@..@@@.@@....@.@..@@@@@@@@.@.@@..@@...@@.@@@...@.@.@@@..@.@.@@@@@@ ...
@.@@@@@@..@@@@...@..@@@@@@.@@@..@.....@@.@.@@...@@@.@@.@.@@@....@@.@.@.@@@@.@@@@@.@@@.@@...@@.@@ ...
.@@@.@.@@@..@@.@.@@@@@.@.@..@@....@..@.@.@@@@.@..@@.@..@@@@@.@@@@@@@.@.@@@.@.@@@.@@@@.@@@@@@@@.@ ...
@@@@@@@.@@...@@@....@.@@@@.@@@@@@@@@.@@@.@@.@@..@...@@@@@.@@@..@.@

### Part 1: How many rolls of paper can be accessed by a forklift?

A roll is **accessible** by forklift if there are fewer than four rolls of paper in the eight adjacent positions. Counting the number of accessible rolls is easy, but I decided to make `accessible rolls` return a list of positions, because I might need that in Part 2.

In [23]:
def accessible_rolls(grid: Grid) -> List[Point]:
    """A roll of paper is accessible if there are fewer than 
    four rolls of paper in the eight adjacent positions."""
    return [p for p in grid if grid[p] == '@'
            if grid.neighbor_contents(p).count('@') < 4]

Here's the answer:

In [24]:
answer(4.1, 1569, lambda:
       len(accessible_rolls(paper_grid)))

Puzzle  4.1:   .0540 seconds, answer 1569            ok

### Part 2: How many rolls of paper can be removed?

If the elves can access a paper roll, they can remove it by forklift. That may make other rolls accessible. How many in total can be removed?

It looks like I was right to make `accessible_rolls` return a list of points rather than a count! I can answer the question by repeatedly finding the accessible rolls, removing them (on a copy of the grid so I don't mess up the original grid), and repeating until there are no more accessible rolls.

In [68]:
def removable_rolls(grid: Grid) -> Iterable[Point]:
    """The positions of paper rolls that can be removed, in any nuber of iterations."""
    grid = grid.copy() # To avoid mutating the input grid
    points = accessible_rolls(grid)
    while points:
        yield from points
        grid.update({p: '.' for p in points})
        points = accessible_rolls(grid)

In [26]:
answer(4.2, 9280, lambda:
       quantify(removable_rolls(paper_grid)))     

Puzzle  4.2:  1.2023 seconds, answer 9280            ok

That's the right answer, but the run time is slow. One issue is that `accessible_rolls` has to look at the whole grid on every iteration. If the previous iteration only removed one or two rolls, that's a waste of time. Instead, we can keep a queue of possibly removable points (initially the points with a paper roll) and repeatedly pop a point off the queue, and if it is an accessible roll, remove it and put all its neighbors on the queue.

In [69]:
def count_removable_rolls(grid1: Grid) -> int:
    """Count the number of paper rolls that can be removed."""
    grid = grid1.copy()    # To avoid mutating the original input grid
    Q = grid.findall('@') # A queue of possibly removable positions in the grid
    while Q:
        p = Q.pop()
        if grid[p] == '@' and grid.neighbor_contents(p).count('@') < 4:
            grid[p] = '.'
            Q.extend(grid.neighbors(p))
    return len(grid1.findall('@')) - len(grid.findall('@')) # The number of '@' removed

In [70]:
answer(4.2, 9280, lambda:
       count_removable_rolls(paper_grid))

Puzzle  4.2:   .1436 seconds, answer 9280            ok

# Summary

In [48]:
for d in sorted(answers):
    print(answers[d])

Puzzle  1.1:   .0008 seconds, answer 1182            ok
Puzzle  1.2:   .0010 seconds, answer 6907            ok
Puzzle  2.1:   .0029 seconds, answer 23560874270     ok
Puzzle  2.2:   .0038 seconds, answer 44143124633     ok
Puzzle  3.1:   .0005 seconds, answer 17085           ok
Puzzle  3.2:   .0022 seconds, answer 169408143086082 ok
Puzzle  4.1:   .0540 seconds, answer 1569            ok
Puzzle  4.2:   .1484 seconds, answer 9280            ok
